In [1]:
# [Cell 1] - Set up paths
import os

is_colab = True
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  BASE_PATH = "/content/drive/MyDrive"  # This will use the current directory
  submission_path = os.path.join(BASE_PATH,  "submission.csv")
  save_model = os.path.join(BASE_PATH,  "siamese.pth")
else:
  BASE_PATH = "/kaggle/input/cvproject"
  submission_path = "submission.csv"
  save_model = "siamese.pth"

num_epochs = 1
batch_size = 2 # power of 2
num_workers = 0 # workers in dataloader
num_train = 2 # number or None
num_test = 1
accumulation_steps = 1
is_tiny_roma = False
# !unzip "/content/drive/MyDrive/cv-22928-2025-a-project.zip" -d "/content/"

project_folder = "cv-22928-2025-a-project/train"
project_folder = os.path.join(BASE_PATH, project_folder)
src = os.path.join(BASE_PATH, "cv-22928-2025-a-project")

test_csv = os.path.join(BASE_PATH, "cv-22928-2025-a-project/test.csv")


if os.path.exists(project_folder):
    print(f"Successfully found project directory at: {project_folder}")
else:
    print(f"ERROR: Could not find project directory at: {project_folder}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Please check if the path '{project_folder}' is correct")

Mounted at /content/drive
Successfully found project directory at: /content/drive/MyDrive/cv-22928-2025-a-project/train


In [2]:
!pip install git+https://github.com/Parskatt/RoMa.git
!pip install tqdm
!pip install torch to pycolmap

  Cloning https://github.com/Parskatt/RoMa.git to /tmp/pip-req-build-5t0jvpr3
  Running command git clone --filter=blob:none --quiet https://github.com/Parskatt/RoMa.git /tmp/pip-req-build-5t0jvpr3
  Resolved https://github.com/Parskatt/RoMa.git to commit edd1b8b35a94e0d6c26c73b1bb3117db8d0b04fa
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 829.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import cv2
import numpy as np
from romatch import roma_outdoor
from tqdm import tqdm
import torch.multiprocessing as mp

mp.set_start_method('spawn', force=True)
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_str)

In [4]:
from romatch import tiny_roma_v1_outdoor, roma_outdoor
if is_tiny_roma:
  roma_model = tiny_roma_v1_outdoor(device=device)
  num_inputs = 15000
else:
  roma_model = roma_outdoor(device=device)
  num_inputs = 30000

Downloading: "https://github.com/Parskatt/storage/releases/download/roma/roma_outdoor.pth" to /root/.cache/torch/hub/checkpoints/roma_outdoor.pth
100%|██████████| 425M/425M [00:02<00:00, 206MB/s]
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_pretrain.pth
100%|██████████| 1.13G/1.13G [00:10<00:00, 120MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using coarse resolution (560, 560), and upsample res (864, 864)


In [5]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import cv2
import numpy as np
from romatch import roma_outdoor
from tqdm import tqdm
import pycolmap


import torch
import torch.nn as nn
import torch.optim as optim

def normalize_frobenius_norm(matrix):
        """Compute the Frobenius norm of a matrix and return the normalized matrix."""
        norm = torch.norm(matrix, p='fro', dim=(-2, -1), keepdim=True)  # Frobenius norm
        normalized_matrix = matrix / (norm + 1e-8)  # Add epsilon to prevent division by zero
        return normalized_matrix

In [6]:
# [Cell 11] - Helper function for matrix flattening
def flatten_matrix(M, num_digits=8):
    """Convert matrix to string format for submission."""
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

def unflatten_matrix(flattened_str):
    """Convert a flattened string back into a 3x3 matrix."""
    # Split the flattened string into a list of values
    values = list(map(float, flattened_str.split()))

    # Ensure the number of values is 9 (for a 3x3 matrix)
    if len(values) != 9:
        raise ValueError("The flattened string must contain exactly 9 values.")

    # Reshape the list of values into a 3x3 matrix
    return np.array(values).reshape(3, 3)

In [7]:
import pandas as pd

df_test = pd.read_csv(test_csv, index_col=False)
df_test["img1_path"] = df_test.apply(lambda row: os.path.join(src, "test_images", row["batch_id"], f"{row['image_1_id']}.jpg"), axis=1)
df_test["img2_path"] = df_test.apply(lambda row: os.path.join(src, "test_images", row["batch_id"], f"{row['image_2_id']}.jpg"), axis=1)
df_test.shape
if num_test is not None:
  df_test = df_test.iloc[:num_train, :]
df_test.shape

(2, 6)

In [8]:





import numpy as np
import torch
import cv2
from torch.utils.data import Dataset

class TestDataset(Dataset):
    def __init__(self, image_pairs, sample_id, transform=None):
        self.image_pairs = image_pairs
        self.sample_id = sample_id
        self.transform = transform

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        imA_path, imB_path = self.image_pairs[idx]
        sample_id = self.sample_id[idx]

        # Get matches and certainty
        warp, certainty = roma_model.match(imA_path, imB_path)
        matches, certainty = roma_model.sample(warp, certainty)

        # Get image dimensions (height, width, channels)
        H_A, W_A, _ = cv2.imread(imA_path).shape
        H_B, W_B, _ = cv2.imread(imB_path).shape

        # Convert to pixel coordinates (RoMa produces matches in [-1,1]x[-1,1])
        kptsA, kptsB = roma_model.to_pixel_coordinates(matches, H_A, W_A, H_B, W_B)

        is_selected = (certainty == certainty.max())
        # Select the keypoints with maximal certainty
        kptsA = kptsA[is_selected]
        kptsB = kptsB[is_selected]
        # print(certainty[is_selected].unique(return_counts=True))



        # Apply cv2.findFundamentalMat only on the selected keypoints
        try:
            F, mask = cv2.findFundamentalMat(
                kptsA.cpu().numpy(),
                kptsB.cpu().numpy(),
                ransacReprojThreshold=1,
                method=cv2.USAC_MAGSAC,
                confidence=0.9999999,
                maxIters=100000
            )

            # If the fundamental matrix is invalid, return a matrix of zeros
            if F is None or F.shape != (3, 3):
                raise ValueError("Invalid fundamental matrix.")

        except Exception as e:
            # Handle exceptions (e.g., invalid input data, computational errors)
            print(f"Error finding fundamental matrix for sample {sample_id}: {e}")
            return torch.zeros((3, 3)).to(device), sample_id  # Return a matrix of zeros in case of error

        # Convert to tensor, normalize Frobenius norm and return
        F_est = torch.tensor(F, dtype=torch.float32).to(device)
        F_est = normalize_frobenius_norm(F_est)

        return F_est.to(device), sample_id

In [12]:
# Example data (replace with real dataset)

image_pairs = [x for x in zip(df_test["img1_path"], df_test["img2_path"])] # Image pairs (replace with your dataset)
sample_id = df_test["sample_id"].values
# Create dataset and dataloader
dataset_test = TestDataset(image_pairs, sample_id, roma_model)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers)

In [13]:



import torch
from tqdm import tqdm

def test_model_with_dataloader(dataloader_test, submission_path, device):
    """
    Perform model inference and save the results (fundamental matrices) to a CSV file.

    Args:
        dataloader_test (DataLoader): PyTorch DataLoader for test samples.
        model (nn.Module): Trained PyTorch model that estimates the fundamental matrix.
        submission_path (str): Path to save the output CSV file.
        device (torch.device): Device where the model is located (CPU or CUDA).
    """

    # List to store results in memory first
    results = []

    # Use torch.no_grad() to disable gradient computation during inference
    with torch.no_grad():
        # Loop through the DataLoader batches
        with torch.cuda.amp.autocast():  # If you're using automatic mixed precision

          for idx_run, (F_est, sample_id) in tqdm(enumerate(dataloader_test)):
            # Make sure the data is on the correct device (GPU/CPU)
            print(idx_run)
            # Loop through the batch and store the result for each sample
            for idx, sample_id_curr in enumerate(sample_id):
                # Flatten the predicted fundamental matrix for output
                flattened_F = flatten_matrix(F_est  [idx])  # Convert matrix to string format
                results.append(f'{sample_id_curr},{flattened_F}\n')  # Add to the results list

    # After processing all the batches, write all results to the CSV file at once
    with open(submission_path, 'w') as f:
        f.write('sample_id,fundamental_matrix\n')  # Write the header
        f.writelines(results)  # Write all the results at once

    print(f"Submission saved to: {submission_path}")


def flatten_matrix(M, num_digits=8):
    """Convert matrix to string format for submission."""
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

In [14]:
test_model_with_dataloader(dataloader_test, submission_path, device)

<ipython-input-13-9285a5361e45>:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # If you're using automatic mixed precision
1it [00:04,  4.68s/it]

0


Submission saved to: /content/drive/MyDrive/submission.csv
